In [ ]:
# MuJoCo, 강화학습 툴, 비디오 저장용 라이브러리 설치

# 1. 시스템 라이브러리 설치 (화면 렌더링에 필요한 드라이버 강제 설치)
!apt-get update && apt-get install -y libgl1-mesa-dev libgl1-mesa-glx libglew-osmesa0 libgl1-mesa-dri

# 2. 환경 변수 설정 (반드시 import mujoco보다 먼저 해야 함!)
import os
os.environ['MUJOCO_GL'] = 'egl'

# 3. 라이브러리 설치
!pip install mujoco gymnasium stable_baselines3 imageio[ffmpeg]

# 4. 라이브러리 임포트 (이제 안전함)
import mujoco
import gymnasium as gym
from stable_baselines3 import PPO
import numpy as np
import imageio

print("설정 완료! 이제 에러가 안 날 겁니다.")

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,573 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,860 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,205 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [69.2 kB]
Get:14 htt

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [1]:
!git clone https://github.com/google-deepmind/mujoco_menagerie.git
# 경로 확인: /content/mujoco_menagerie/unitree_go1/scene.xml

Cloning into 'mujoco_menagerie'...
remote: Enumerating objects: 6448, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 6448 (delta 85), reused 58 (delta 58), pack-reused 6306 (from 3)
Receiving objects: 100% (6448/6448), 475.73 MiB | 15.05 MiB/s, done.
Resolving deltas: 100% (2148/2148), done.
Updating files: 100% (2427/2427), done.


In [2]:
import gymnasium as gym
import mujoco
import numpy as np
from gymnasium import spaces

class UnitreeGo1Env(gym.Env):
    def __init__(self):
        super().__init__()
        # 1. 모델 로드
        self.model = mujoco.MjModel.from_xml_path("/content/mujoco_menagerie/unitree_go1/scene.xml")
        self.data = mujoco.MjData(self.model)

        # 2. Action Space 정의 (12개 관절 토크 제어)
        # 값의 범위는 -1 ~ 1 (나중에 스케일링)
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(12,), dtype=np.float32)

        # 3. Observation Space 정의 (관절 각도 + 속도 + 자이로 등)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(30,), dtype=np.float32)

    def reset(self, seed=None):
        mujoco.mj_resetData(self.model, self.data)
        # 초기 상태 반환
        return self._get_obs(), {}

    def step(self, action):
        # ... (이전 코드: action 적용 및 물리 스텝 진행) ...

        # -----------------------------------------------------------
        # [데이터 수집]
        # -----------------------------------------------------------
        # 1. 목표 속도 (예: 앞으로 1.0 m/s 가라)
        target_velocity = 1.0

        # 2. 현재 상태 가져오기
        velocity_x = self.data.qvel[0] # 전진 속도
        base_ang_vel = self.data.qvel[3:6] # 몸통 회전 속도 (Roll, Pitch, Yaw)

        # 3. 관절 상태
        # qpos[7:]가 다리 관절입니다. (Go1은 다리당 3개 * 4 = 12개)
        # default_joint_pos는 로봇이 예쁘게 서 있는 자세의 각도값입니다.
        # (Go1의 경우 대략 [0, 0.9, -1.8] * 4 형태입니다. 값을 모르면 일단 0으로 둡니다)
        current_joint_pos = self.data.qpos[7:]
        default_joint_pos = np.array([0.0, 0.9, -1.8] * 4)

        # -----------------------------------------------------------
        # [보상 함수 계산] (Reward Engineering)
        # -----------------------------------------------------------

        # 1. Tracking Reward (목표 속도 따라가기)
        # 속도 차이가 작을수록 점수가 높음 (Exponential kernel 사용 추천)
        v_err = target_velocity - velocity_x
        rew_tracking = np.exp(-5.0 * (v_err ** 2))

        # 2. Stability Reward (자세 제어)
        # 몸통 각속도(ang_vel)가 0에 가까워야 함 (흔들림 방지)
        rew_lin_vel_z = -np.square(self.data.qvel[2]) # 위아래로 방방 뛰지 마
        rew_ang_vel = -np.sum(np.square(base_ang_vel[:2])) # Roll, Pitch 흔들림 벌점

        # 3. Smoothness Reward (부드러운 움직임)
        # 관절이 기본 자세에서 얼마나 비틀어졌나?
        rew_dof_pos = -np.sum(np.square(current_joint_pos - default_joint_pos))
        # 토크를 얼마나 썼나?
        rew_torque = -np.sum(np.square(action))

        # -----------------------------------------------------------
        # [최종 합산] (가중치 튜닝이 실력!)
        # -----------------------------------------------------------
        total_reward = (
            1.0  * rew_tracking +
            0.5  * rew_lin_vel_z +
            0.05 * rew_ang_vel +
            0.1  * rew_dof_pos +
            0.005 * rew_torque
        )

        # 넘어짐 벌점
        terminated = False
        if self.data.qpos[2] < 0.2:
            terminated = True
            total_reward -= 10.0

        return self._get_obs(), total_reward, terminated, False, {}

    def _get_obs(self):
        # 현재 관절 각도와 속도를 합쳐서 리턴
        return np.concatenate([self.data.qpos[7:], self.data.qvel[6:]]).astype(np.float32)

    def close(self):
        pass

ModuleNotFoundError: No module named 'gymnasium'

In [5]:
from stable_baselines3 import PPO
import numpy as np
import types

# 환경 생성
env = UnitreeGo1Env()

# Fix: Monkey-patch the _get_obs method to return 30 elements
def _get_obs_fixed(self):
    return np.concatenate([
        self.data.qpos[7:],    # 12 joint positions
        self.data.qvel[0:3],   # 3 base linear velocities
        self.data.qvel[3:6],   # 3 base angular velocities
        self.data.qvel[6:]     # 12 joint velocities
    ]).astype(np.float32)

env._get_obs = types.MethodType(_get_obs_fixed, env)

# Fix: Downgrade sympy to a version compatible with torch/stable_baselines3
!pip install sympy==1.12

# PPO 모델 생성 (MlpPolicy: 이미지 말고 수치 데이터 사용)
model = PPO("MlpPolicy", env, verbose=1)

# 학습 시작 (Colab 무료 버전 기준 10만~100만 스텝 추천)
model.learn(total_timesteps=700000)

# 모델 저장
model.save("ppo_unitree_go1")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 43.4 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.14.0
    Uninstalling sympy-1.14.0:
      Successfully uninstalled sympy-1.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.9.0+cpu requires sympy>=1.13.3, but you have sympy 1.12 which is incompatible.
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 1456 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [4]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install pyvirtualdisplay > /dev/null 2>&1

In [14]:
import imageio
import numpy as np
import gymnasium as gym
import types
import time # 시간 측정을 위해 추가

# --- 1. 렌더링 함수 (그대로) ---
def _render_fixed(self, mode="rgb_array", width=640, height=480):
    if not hasattr(self, '_renderer'):
        self._renderer = mujoco.Renderer(self.model, height=height, width=width)
        self._camera = mujoco.MjvCamera()
        mujoco.mjv_defaultFreeCamera(self.model, self._camera)
        self._camera.distance = 3.0
        self._camera.elevation = -30
        self._camera.azimuth = 90

    mujoco.mj_forward(self.model, self.data)
    robot_pos = self.data.qpos[:3]
    self._camera.lookat[:] = robot_pos

    self._renderer.update_scene(self.data, camera=self._camera)
    return self._renderer.render()

env.render = types.MethodType(_render_fixed, env)

# --- 2. 영상 녹화 루프 (진행상황 표시 추가) ---
print(f"🎬 영상 녹화를 시작합니다... (목표: {300} 스텝)")

frames = []
obs, info = env.reset()
max_steps = 300

start_time = time.time() # 시작 시간 기록

for i in range(max_steps):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)

    img = env.render()
    frames.append(img)

    # ★ 여기가 추가된 부분: 50번마다 진행상황 출력
    if (i + 1) % 50 == 0:
        elapsed = time.time() - start_time
        print(f"📸 진행 중: {i + 1}/{max_steps} 프레임 완료 ({elapsed:.1f}초 경과)")

    if terminated or truncated:
        print(f"🛑 에피소드 종료 (Step: {i})")
        break

env.close()

# --- 3. 저장 ---
print("💾 파일 저장 중...")
if len(frames) > 0:
    imageio.mimsave('robot_video.mp4', frames, fps=30)
    print("✅ 'robot_video.mp4' 저장 완료!")
else:
    print("⚠️ 저장된 프레임이 없습니다.")

# 재생
try:
    from IPython.display import Video
    display(Video("robot_video.mp4", embed=True))
except:
    pass

🎬 영상 녹화를 시작합니다... (목표: 300 스텝)
📸 진행 중: 50/300 프레임 완료 (83.1초 경과)
📸 진행 중: 100/300 프레임 완료 (161.3초 경과)
📸 진행 중: 150/300 프레임 완료 (236.4초 경과)
📸 진행 중: 200/300 프레임 완료 (313.0초 경과)
📸 진행 중: 250/300 프레임 완료 (388.0초 경과)
📸 진행 중: 300/300 프레임 완료 (464.6초 경과)
💾 파일 저장 중...
✅ 'robot_video.mp4' 저장 완료!
